In [33]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as np
import random

In [34]:
path = keras.utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
with open(path, 'r', encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace('\n', ' ')
print("Courpus lens: ", len(text))

chars = sorted(list(set(text)))
print("Total chars: ", len(chars))
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))

Courpus lens:  600893
Total chars:  56


In [35]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print("Number of Seq: ", len(sentences))

Number of Seq:  200285


In [36]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1

In [37]:
model = Sequential([
    keras.Input(shape=(maxlen, len(chars))),
    layers.LSTM(128),
    layers.Dense(len(chars), activation="softmax")
])
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [38]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [41]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char2id[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = id2char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

1565/1565 [==============================] - 78s 50ms/step - loss: 1.4808

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "psychologist is about to put an end to t"
...Generated:  he power and the proper the religion and expensible and sense of the proper the self-respect of the superforered and such as the sense of the religion of the more the sense of the life of the senses of the spirit of the personal consequences of the self-respection of the self-rears of the properity and senses of the soul of the spirit of the great constitution of the superforerer of the great mora

...Diversity: 0.5
...Generating with seed: "psychologist is about to put an end to t"
...Generated:  he superferent as in the superfects that the consequence of moral the spirition of the spirition mean in the simplation, that are the moral or and respect and other as strumble to the tertion, and the consequence of the prolt most the soul and for itself and former of an assign that the li

<ipython-input-38-ded0137f8096>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:  hs?--that and very or the remainy, as that need ourselves as had suclocttize the persand congeinisasing will as always to the saint: mranded and madfed state. such meany concerning not he explo: them? as a strong to be twranchless finliness peanks aoblige, elove differeness, modes of the consciousness against another himself in new music dis lapks of a stand by by freedom of test nimil a common th

...Diversity: 1.2
...Generating with seed: " who art thou? i see thee follow thy pat"
...Generated:  ies, and matten was cesstafiss--who was have formedles: another them contemple, such ya from the sculisabulity nair, while influence disle: yoo disintsiptm against anveia of requirationesment, from the centurious their wain one nei? how saids   hood ages by the hoping is a addly doot that considers of a tiving, so mere rateperomians, "wiste"--degenduarary to know he ws lavers is excinism.utive ent

1565/1565 [==============================] - 82s 53ms/step - loss: 1.2425

Gener